In [1]:
import sys
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.preprocessing import PolynomialFeatures, LabelEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import Lasso, Ridge
import joblib
from joblib import dump

In [2]:
df_data=pd.read_csv("./data/all_tables_combined.csv")

FileNotFoundError: [Errno 2] No such file or directory: './data/all_tables_combined.csv'

In [ ]:
df_data.head(3)

In [ ]:
#df_s_parent=df_data[df_data['HH_COMP_DESC']=="1 Adult Kids"]
df_s_parent=df_data

In [ ]:
df_s_parent['NUM_DAYS']=df_s_parent['END_DAY'] - df_s_parent['START_DAY'] + 1
df_s_parent.drop(['START_DAY','END_DAY','COUPON_UPC','household_key','PRODUCT_ID'], axis=1, inplace=True)

In [ ]:
df_s_parent.head(2)

In [ ]:
#sns.pairplot(df_s_parent)

In [ ]:
df_s_parent

In [ ]:
df_s_parent['DISCOUNT_PCT']=(df_s_parent['RETAIL_DISC']+df_s_parent['COUPON_DISC']+df_s_parent['COUPON_MATCH_DISC'])*100/(df_s_parent['RETAIL_DISC']+df_s_parent['COUPON_DISC']+df_s_parent['COUPON_MATCH_DISC']+df_s_parent['SALES_VALUE'])

In [ ]:
df_s_parent['description'].value_counts()

In [ ]:
df_s_parent['SUB_COMMODITY_DESC'].value_counts().head(10)

In [ ]:
df_s_parent[df_s_parent['SUB_COMMODITY_DESC'].str.contains('MILK')]['SUB_COMMODITY_DESC'].unique()

In [ ]:
df_s_parent=df_s_parent[(df_s_parent['SUB_COMMODITY_DESC']=='FLUID MILK WHITE ONLY') | (df_s_parent['SUB_COMMODITY_DESC']=='MISCELLANEOUS MILK') | (df_s_parent['SUB_COMMODITY_DESC']=='CHOCOLATE MILK')]

In [ ]:
df_s_parent.columns.values

In [ ]:
df_s_parent.drop(['TRANS_TIME','AGE_DESC', 'MARITAL_STATUS_CODE', 'INCOME_DESC', 'HOMEOWNER_DESC', 'HH_COMP_DESC', 'HOUSEHOLD_SIZE_DESC','KID_CATEGORY_DESC', 'DEPARTMENT', 'COMMODITY_DESC'], axis=1, inplace=True)

In [ ]:
df_s_parent

In [ ]:
df_s_parent['CURR_SIZE_OF_PRODUCT'].value_counts()

In [ ]:
def label_to_volume(label):
    volume=int()
    if label=='1 GA':
        volume=128
    elif label=='1 PT':
        volume=16
    elif label=='16 OZ':
        volume=16
    elif label=='1 QT':
        volume=32
    elif label=='PT':
        volume=16
        
    return volume

vect_label_to_volume=np.vectorize(label_to_volume)

In [ ]:
df_s_parent['VOLUME']=vect_label_to_volume(df_s_parent['CURR_SIZE_OF_PRODUCT'])
df_s_parent.head()

In [ ]:
df_s_parent.groupby('CAMPAIGN').sum()

In [ ]:
df_s_parent=df_s_parent[df_s_parent['SALES_VALUE']<=8]

In [ ]:
sales=df_s_parent['SALES_VALUE']

In [ ]:
df_s_parent_features=df_s_parent.drop('SALES_VALUE', axis=1)

In [ ]:
dummy=pd.get_dummies(df_s_parent_features['BRAND'],drop_first=True)
dummy_milk_type=pd.get_dummies(df_s_parent_features['SUB_COMMODITY_DESC'],drop_first=True)
#dummy_hh_size_desc=pd.get_dummies(df_s_parent_features['HOUSEHOLD_SIZE_DESC'],drop_first=True)
dummy_manufacturer=pd.get_dummies(df_s_parent_features['MANUFACTURER'],drop_first=True)
df_s_parent_features=pd.concat([df_s_parent_features,dummy], axis=1)
df_s_parent_features=pd.concat([df_s_parent_features,dummy_milk_type], axis=1)
#df_s_parent_features=pd.concat([df_s_parent_features,dummy_hh_size_desc], axis=1)
df_s_parent_features=pd.concat([df_s_parent_features,dummy_manufacturer], axis=1)
df_s_parent_features.columns

In [ ]:
df_s_parent_features.drop(['STORE_ID','QUANTITY','BRAND','description','MANUFACTURER','CURR_SIZE_OF_PRODUCT','SUB_COMMODITY_DESC', 'CAMPAIGN', 'NUM_DAYS'], axis=1, inplace=True)

In [ ]:
sns.heatmap(df_s_parent_features.corr())

In [ ]:
df_s_parent_features.columns.values

In [ ]:
X=df_s_parent_features
y=sales

In [ ]:
X_train,X_test,y_train,y_test=train_test_split(X, y, random_state = 22)

In [ ]:
sscaler=StandardScaler()

In [ ]:
X_train_scaled=sscaler.fit_transform(X_train)
dump(sscaler,'scaler_file.save')

X_test_scaled=sscaler.fit_transform(X_test)

In [ ]:
lasso=Lasso(alpha=0.00001,max_iter=99999)
lasso.fit(X_train_scaled,y_train)

In [ ]:
y_preds=lasso.predict(X_test_scaled)
plt.scatter(y_test,y_preds)

In [ ]:
sns.regplot(y_test,y_preds)

In [ ]:
features_and_coefs=pd.DataFrame({'features':X.columns, 'coefs':lasso.coef_}).sort_values(by='coefs', ascending=False)
features_and_coefs.features.values

In [ ]:
X_test_scaled[0]

In [ ]:
cross_val_score(lasso, X_train_scaled,y_train)

In [ ]:
cross_val_score(lasso, X_test_scaled,y_test)

Next let's take the columns whose coefficients have a magnitude over 0.01. We'll try to estimate using ridge instead of lasso and see hot it performs.

NB: REMOVE THE UNECESSARY COLUMNS FROM THE FEATURE MATRIX. NOT DONE YET.

In [ ]:
ridge=Ridge()
ridge.fit(X_train_scaled,y_train)

In [ ]:
y_ridge_preds=ridge.predict(X_test_scaled)

In [ ]:
cross_val_score(ridge, X_test_scaled,y_test)

In [ ]:
plt.scatter(y_test,y_ridge_preds)

In [ ]:
dump(lasso, 'spf_model.joblib')